## Import data

In [1]:
import networkx as nx
import json
filename="Full_event.json"
with open(filename) as jsonfile:
    data = json.load(jsonfile)

In [3]:
filename="Users.json"
with open(filename, "r", encoding='utf8') as jsonfile:
    user_data = json.load(jsonfile)

## User --> Image network (edge weight based on slider)

In [7]:
user_dict={}

for user in user_data["result"]["users"]:
    user_dict[user["id"]]=user


edges=[]
graph=nx.DiGraph()

def convert(slider):
    if slider==1:
        edge_weight_1=2
        edge_weight_2=-2
    if slider==2:
        edge_weight_1=1
        edge_weight_2=-1
    if slider==3:
        edge_weight_1=0
        edge_weight_2=0
    if slider==4:
        edge_weight_1=1
        edge_weight_2=1
    
    if slider==5:
        edge_weight_1=2
        edge_weight_2=2
    return(edge_weight_1,edge_weight_2)

photo_dict={}
#user_dict={}
count=0
for photo in data["result"]["photos"]:
    #print(photo["annotationAnswers"])
    
    photo_id=photo["id"]
    photo_url=photo["imageUrl"]
    photo_time=photo['createdAt']
    if photo["locationData"] is not None:
        photo_lat=photo["locationData"]["latitude"]
        photo_lng=photo["locationData"]["longitude"]
    else:
        photo_lat = ''
        photo_lng = ''
   
        
    photo_dict[photo_id]={"photo_url":photo_url, "latitude":photo_lat, "longitude":photo_lng, "photo_time":photo_time}
    if not photo["annotationAnswers"]:
        count+=1
        continue
    owner_id=photo["createdBy"]
    owner_slider=photo["annotationAnswers"][0]["sliderAnswer"]

    edge_weight_1,edge_weight_2=convert(owner_slider)
    if not edge_weight_1==0:
        edge=(owner_id,photo_id,{"weight":edge_weight_1,"real_weight":edge_weight_2,"relation_type":"owner"})
        edges.append(edge)
    for reaction in photo["reactions"]:
        reactor_id=reaction["createdBy"]
        
        if reactor_id not in user_dict:
            user_dict[owner_id]={"phone":'not captured'}
            
        reactor_slider=reaction["answers"][0]["sliderAnswer"]
        edge_weight_1,edge_weight_2=convert(reactor_slider)
        if not edge_weight_1==0:
            edge=(reactor_id,photo_id,{"weight":edge_weight_1,"real_weight":edge_weight_2,"relation_type":"reaction"})
            edges.append(edge)
            
graph.add_edges_from(edges)
        
for node in graph.nodes():
    if node in photo_dict:
        graph.nodes[node]["type"]="image"
        graph.nodes[node]["url"]=photo_dict[node]["photo_url"]
        graph.nodes[node]["time"]=photo_dict[node]["photo_time"]
        graph.nodes[node]["latitude"]=photo_dict[node]["latitude"]
        graph.nodes[node]["longitude"]=photo_dict[node]["longitude"]
        graph.nodes[node]["phone_type"]="na"

    #if node in user_dict:
    
    else:
        graph.nodes[node]["type"]="user"
        if node in user_dict:
            try:
                graph.nodes[node]["demo_group"]=user_dict[node]["demographicGroup"]
            except KeyError:
                graph.nodes[node]["demo_group"]="na"
        else:
            graph.nodes[node]["demo_group"]="na"
            
import numpy as np
for photo in data["result"]["photos"]: 
    slider_answers = []
    photo_id=photo["id"]
    if not photo["annotationAnswers"]:
        continue
    for owner_answer in photo["annotationAnswers"]: 
        if owner_answer['sliderAnswer'] is not None:
            slider_answers.append(owner_answer['sliderAnswer'])
     
    for reaction in photo["reactions"]:
        for position in (reaction['answers']): 
            if position['sliderAnswer'] is not None:
                slider_answers.append(position['sliderAnswer'])
                
    variance=np.var(slider_answers)
    #print("variance is",variance)
    photo_dict[photo_id]["variance"]=variance 
    if photo_id in graph.nodes:
        graph.nodes[photo_id]["variance"]=float(variance)
nx.write_gexf(G=graph,path="test_network_UB_slider_test.gexf")

## User 2 tag network on a global level

In [8]:
import networkx as nx
import json
user_dict={}

for user in user_data["result"]["users"]:
    user_dict[user["id"]]=user

graph=nx.DiGraph()


edges=[]
for photo in data["result"]["photos"]:
    #print(photo["annotationAnswers"])
        
    owner_id=photo["createdBy"]
    if not photo["annotationAnswers"]:
        continue
    owner_tags=photo["annotationAnswers"][1]["multipleChoiceAnswer"]
    if owner_id in user_dict:
        
        if "tags" in user_dict[owner_id]:
            user_dict[owner_id]["tags"]+=owner_tags
        else:
            user_dict[owner_id]["tags"]=owner_tags
           
        if "no_reactions" in user_dict[owner_id]:
            user_dict[owner_id]["no_reactions"]+=1
        else:
            user_dict[owner_id]["no_reactions"]=1
            
            
    for reaction in photo["reactions"]:
        reactor_id=reaction["createdBy"]
        reactor_tags=reaction["answers"][1]["multipleChoiceAnswer"]       
        if reactor_id in user_dict:
            if "tags" in user_dict[reactor_id]:
                user_dict[reactor_id]["tags"]+=reactor_tags
            else:
                user_dict[reactor_id]["tags"]=reactor_tags
            
            if "no_reactions" in user_dict[reactor_id]:
                user_dict[reactor_id]["no_reactions"]+=1
            else:
                user_dict[reactor_id]["no_reactions"]=1

        
from collections import Counter
for user in user_dict:
    if "no_reactions" in user_dict[user]:
        no_reactions=user_dict[user]["no_reactions"]
        cnt=Counter(user_dict[user]["tags"]).most_common(20)
        for each in cnt:
            
            raw_weight=each[1]
            n_weight=raw_weight/no_reactions

            edge=(user,each[0],{"raw_weight":raw_weight,"normalized_weight":n_weight})
            edges.append(edge)
graph.add_edges_from(edges)
   
    
for node in graph.nodes():
    if node in user_dict:
        graph.nodes[node]["type"]="user"
        graph.nodes[node]["demo_group"]=user_dict[node]["demographicGroup"]
    else: 
        graph.nodes[node]["type"]="tag"
        graph.nodes[node]["tag"]=node

    
nx.write_gexf(G=graph,path="test_network_UB_tags_test.gexf")
    
